<a href="https://colab.research.google.com/github/mohanrajmit/AgeCalculation/blob/master/AgeEstimation_MobileNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!git clone https://github.com/mohanrajmit/AgeCalculation.git

fatal: destination path 'AgeCalculation' already exists and is not an empty directory.


Age Estimation for Indian Movie Face Dataset

In [1]:
# keras imports

from keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
from keras.models import Model
from keras.models import model_from_json
from keras.layers import Input
from keras.applications import vgg16
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import cv2
import h5py
import os
import json
import datetime
import time
import pandas as pd






In [2]:
data=pd.read_csv("/content/AgeCalculation/dataset/train.csv")

image_name = data["ID"]
label_name = data["Class"]

features = []
labels = []

In [3]:
data.shape

(2435, 2)

In [4]:
#from tensorflow.keras.applications.inception_v3 import InceptionV3,preprocess_input
#from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2,preprocess_input


In [5]:

pretrained_nn = MobileNetV2(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
#pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
#pretrained_nn = InceptionV3(weights='imagenet',pooling = 'avg',include_top=False, input_shape=(224, 224, 3))

#pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
image_size = (224, 224)

In [6]:
dataset_path="/content/AgeCalculation/dataset/Train"

In [8]:
labels=[]
images_lst=[]
for image_name1,label in zip(image_name,label_name):	
	print("the image name  {} and label{}" .format(image_name1,label))
	image_path= str(dataset_path+"/"+image_name1)
	labels.append(label)	#print(image_path)


	img = load_img(image_path, target_size=image_size)
	x = img_to_array(img)
	images_lst.append(x)




the image name  377.jpg and labelMIDDLE
the image name  17814.jpg and labelYOUNG
the image name  21283.jpg and labelMIDDLE
the image name  16496.jpg and labelYOUNG
the image name  4487.jpg and labelMIDDLE
the image name  6283.jpg and labelMIDDLE
the image name  23495.jpg and labelYOUNG
the image name  7100.jpg and labelYOUNG
the image name  6028.jpg and labelYOUNG
the image name  22617.jpg and labelOLD
the image name  11177.jpg and labelYOUNG
the image name  2462.jpg and labelMIDDLE
the image name  24116.jpg and labelMIDDLE
the image name  17689.jpg and labelMIDDLE
the image name  728.jpg and labelMIDDLE
the image name  3003.jpg and labelMIDDLE
the image name  14408.jpg and labelOLD
the image name  6679.jpg and labelYOUNG
the image name  15152.jpg and labelOLD
the image name  24784.jpg and labelMIDDLE
the image name  9970.jpg and labelYOUNG
the image name  22550.jpg and labelOLD
the image name  150.jpg and labelYOUNG
the image name  7379.jpg and labelMIDDLE
the image name  15387.jpg an

In [9]:
# Create a single numpy array with all the images we loaded
x_train = np.array(images_lst)

# Also convert the labels to a numpy array
y_train = np.array(labels)

# Normalize image data to 0-to-1 range
x_train = preprocess_input(x_train)

# Load a pre-trained neural network to use as a feature extractor
#model = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))
#pretrained_nn = vgg16.VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Extract features for each image (all in one pass)
features_x = pretrained_nn.predict(x_train)




77/77 [==============================] - 10s 36ms/step


In [10]:
features_x.shape

(2435, 7, 7, 1280)

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from pathlib import Path
import joblib
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import confusion_matrix

In [12]:
# encode the labels using LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
from keras.utils import np_utils
y_train = np_utils.to_categorical(y_train, 3)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(features_x, y_train, test_size=0.20, random_state=42)



In [14]:
X_train.shape

(1948, 7, 7, 1280)

In [15]:
y_train[0]

array([1., 0., 0.], dtype=float32)

In [16]:
# Create a model and add layers
model = Sequential()

model.add(Flatten(input_shape=X_train.shape[1:]))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=['accuracy']
)



In [17]:
#class_names = ["not_glaucoma","glaucoma"]
# Train the model
H = model.fit(X_train,y_train,validation_data=(X_test, y_test),epochs=50,shuffle=True)

Epoch 1/50
61/61 [==============================] - 1s 15ms/step - loss: 8.2215 - accuracy: 0.5149 - val_loss: 0.9826 - val_accuracy: 0.5832
Epoch 2/50
61/61 [==============================] - 1s 10ms/step - loss: 0.8694 - accuracy: 0.6129 - val_loss: 0.8519 - val_accuracy: 0.6222
Epoch 3/50
61/61 [==============================] - 1s 10ms/step - loss: 0.7981 - accuracy: 0.6304 - val_loss: 0.8610 - val_accuracy: 0.6345
Epoch 4/50
61/61 [==============================] - 1s 10ms/step - loss: 0.7135 - accuracy: 0.6561 - val_loss: 0.8638 - val_accuracy: 0.6242
Epoch 5/50
61/61 [==============================] - 1s 10ms/step - loss: 0.6655 - accuracy: 0.6915 - val_loss: 0.8289 - val_accuracy: 0.6489
Epoch 6/50
61/61 [==============================] - 1s 10ms/step - loss: 0.5980 - accuracy: 0.7254 - val_loss: 0.8454 - val_accuracy: 0.6304
Epoch 7/50
61/61 [==============================] - 1s 11ms/step - loss: 0.5562 - accuracy: 0.7315 - val_loss: 0.8412 - val_accuracy: 0.6366
Epoch 8/50
61